In [1]:
%load_ext lab_black
%cd ..

/root/dev/ddpm/SDF-Diffusion


In [51]:
from pathlib import Path

import torch as th
import numpy as np
import yaml
from easydict import EasyDict

from src.utils import instantiate_from_config
from src.utils.vis import save_sdf_as_mesh

In [12]:
th.set_grad_enabled(False)
th.cuda.set_device(4)

# Load Pretrained Models

In [30]:
args_path = "config/gen32/shapenet_airplane.yaml"
ckpt_path = "results/gen32/airplane.pth"

In [31]:
with open(args_path) as f:
    args = EasyDict(yaml.safe_load(f))

In [32]:
model = instantiate_from_config(args.model).cuda()
ckpt = th.load(ckpt_path, map_location="cpu")
model.load_state_dict(ckpt["model_ema"])

In [42]:
ddpm_sampler = instantiate_from_config(args.ddpm.valid).cuda()

In [47]:
preprocessor = instantiate_from_config(args.preprocessor, "cuda")

# Generate New Data

In [43]:
out = ddpm_sampler.sample_ddim(model, (1, 1, 32, 32, 32), show_pbar=True)

Sample DDIM: 100%|██████████████████████████████████████████████████████████████████████████████| 50/50 [00:04<00:00, 11.88it/s]


In [50]:
out = preprocessor.destandardize(out)

## Save as an OBJ file

In [52]:
save_sdf_as_mesh("test.obj", out, safe=True)

TODO: add upsampling here